# Evaluation of default configurations


We have two answer to questions:
 1. By which method can we find good Symbolic Defaults? 
 2. **Can we find good (i.e. better than currently known) symbolic defaults?**
 
This notebook addresses the second question.

----

general remarks:
 - does not currently factor in runtime, but this may be especially important for default values. I didn't explicitly measure runtime, but the 'symbolic defaults' in adaboost take much longer (because it significantly increases `n_estimators` and `max_depth`).

In [10]:
from persistence import load_problem, load_results_for_problem
from visualization.output_parser import get_performance_from_console_output, get_performance_from_csv

def load_random_search_results(problem_name):
    p = load_problem('problems.json', problem_name)
    return load_results_for_problem(p)

# 1a. SVC

After determining good symbolic defaults, we ought to see how they compare to current (scikit-learn) defaults. To this end, we compare three different default configurations (in bold is the name by which they will be referenced henceforth):

 - The **symbolic_pre** defaults we found from evolutionary optimization, specifically: `C=128, gamma=(mkd / 4)`. 
 This symbolic function uses metafeatures as calculated on the dataset *before* it is preprocessed.
 - The **symbolic_post** defaults we found from evolutionary optimization, specifically: `C=64, gamma=mkd`.
     This symbolic function uses metafeatures as calculated on the dataset *after* it has been preprocessed.
 - The scikit-learn **0.20** defaults, specifically: `C=1., gamma=(1 / n_features)`
 - The scikit-learn >= **0.22** defaults, specifically: `C=1., gamma=(1 / (n_features * X.var()))`
 
Note that actually all of these defaults are symbolic.

A second important detail to note is that these settings are not tried by themselves.
A (fairly standard) preprocessing pipeline is applied:
 - **Imputation**: using the mean for numeric features, and the most frequent value for categorical features.
 - **Transformation**: numeric features are scaled to N(0, 1), categorical features are one-hot encoded.
 - **Feature Selection**: all constant features are removed.
 
After these steps, the SVC is invoked on the preprocessed data with the given values for `C` and `gamma`.

Note: for the scikit-learn defaults, currently the metafeatures of the preprocessed data are used (e.g. `n_features` is determined after one-hot encoding, for instance). For the *symbolic* method `mkd` is determined on the original, (largely) unprocessed dataset (samples with NaN values are ignored).

## 1a.1  Loading Data

In [11]:
# The grid search result from Jan.
svc_results = load_random_search_results('svc_rbf')

# results currently still stored in log. should be aggregated to single file..
# "data/results/pipeline_c128mkd4.txt"
symb_default_performances = get_performance_from_csv("data/results/ppp_svc_rbf_C32_Gmkd.txt")
old_default_performances = get_performance_from_csv("data/results/ppp_svc_rbf_default.txt")
new_default_performances = get_performance_from_csv("data/results/ppp_svc_rbf_default_Gscale.txt")

## 1a.2 Comparing Results
We compare results by number of times one's average cross-validation performance is better (first three columns) and by their loss as compared to the best found result in the original set of experiments ('loss' column).

**note:** The 'loss' column is calculated over all executed tasks with the default, because of work-in-progress/earlier cut-off with time constaints, currently the amount of tasks evaluated per method differs, so you find the amount of tasks for which the defaults have been evaluated in the 'N' column. The total loss is *not* normalized for the amount of completed tasks.

In [12]:
import pandas as pd
import numpy as np

methods = ['Symbolic', '0.20', '0.22']
df = pd.DataFrame(np.zeros(shape=(len(methods), len(methods)+2)), columns = methods + ['loss', 'N'])
df.index = methods

# Calculate 'wins'
performances = list(zip(methods, [symb_default_performances, old_default_performances, new_default_performances]))
for (method, performance) in performances:
    for (method2, performance2) in performances:
        one_over_two = (performance.avg - performance2.avg) > 0
        df.loc[method][method2] = sum(one_over_two)

# Calculate loss        
for (method, performance) in performances:
    loss_sum = 0
    for i, row in performance.iterrows():
        best_score = svc_results[svc_results.task_id == row.name].predictive_accuracy.max()
        loss = best_score - row.avg
        if loss < 0:
            print('{} outperformed best on task {} by {}'.format(method, row.name, loss))
        loss_sum += loss
    df.loc[method]['loss'] = loss_sum
    df.loc[method]['N'] = len(performance)
    
df

Symbolic outperformed best on task 12 by -1.1102230246251565e-16


,Symbolic,0.20,0.22,loss,N
Symbolic,0.0,67.0,65.0,1.882626,92.0
0.20,19.0,0.0,3.0,4.107663,92.0
0.22,21.0,26.0,0.0,3.493432,92.0


This reads as *Symbolic* won over the *0.20* default 67 times, while the *0.20* default was better than *Symbolic* on 19 tasks. *Symbolic* obtained a loss of 1.74 over the best known result of each task (or slightly under 0.02 accuracy, on average).

We see that *Symbolic* (i.e. `C=128, gamma=mkd/4`) as default outperforms either of the two scikit-learn ones, both in terms of tasks where it achieves higher predictive accuracy, and the loss in accuracy it occurs across tasks.

# 1b SVC Poly

## 1b.1 Loading Data

In [13]:
# The grid search result from Jan.
svc_results = load_random_search_results('svc_poly')

# results currently still stored in log. should be aggregated to single file..
# "data/results/pipeline_c128mkd4.txt"
symb_default_performances = get_performance_from_csv("data/results/ppp_svc_poly_D1_C6_G7.txt")
old_default_performances = get_performance_from_csv("data/results/ppp_svc_poly_default.txt")

## 1b.2 Comparing Results
We compare results by number of times one's average cross-validation performance is better (first three columns) and by their loss as compared to the best found result in the original set of experiments ('loss' column).

**note:** The 'loss' column is calculated over all executed tasks with the default, because of work-in-progress/earlier cut-off with time constaints, currently the amount of tasks evaluated per method differs, so you find the amount of tasks for which the defaults have been evaluated in the 'N' column. The total loss is *not* normalized for the amount of completed tasks.

In [14]:
import pandas as pd
import numpy as np

methods = ['Symbolic', '0.20']
df = pd.DataFrame(np.zeros(shape=(len(methods), len(methods)+2)), columns = methods + ['loss', 'N'])
df.index = methods

# Calculate 'wins'
performances = list(zip(methods, [symb_default_performances, old_default_performances]))
for (method, performance) in performances:
    for (method2, performance2) in performances:
        one_over_two = (performance.avg - performance2.avg) > 0
        df.loc[method][method2] = sum(one_over_two)

# Calculate loss        
for (method, performance) in performances:
    loss_sum = 0
    for i, row in performance.iterrows():
        best_score = svc_results[svc_results.task_id == row.name].predictive_accuracy.max()
        loss = best_score - row.avg
        if loss < 0:
            print('{} outperformed best on task {} by {}'.format(method, row.name, loss))
        loss_sum += loss
    df.loc[method]['loss'] = loss_sum
    df.loc[method]['N'] = len(performance)
    
df

0.20 outperformed best on task 9971 by -8.721690845203689e-06
0.20 outperformed best on task 9971 by -8.721690845203689e-06


,Symbolic,0.20,loss,N
Symbolic,0.0,12.0,12.004243,92.0
0.20,77.0,0.0,3.887059,91.0


# 2. AdaBoost

From the evolutionary optimization, we found that often recommended symbolic defaults:
 - **learning rate**: 0.75..1.0
 - **n_estimators**: n
 - **max_depth**: p
 
However, as also noted due to the values of hyperparameters in the original experiments, it might as well read `n_estimators=500` and `max_depth=10`, the bounds of the original experiments.

I am not aware of any proposed changes to default values for this algorithm, so the only comparison is to scikit-learn defaults as they are.

In [15]:
# The grid search result from Jan.
adaboost_results = load_random_search_results('adaboost')

# results currently still stored in log. should be aggregated to single file..
# "data/results/pipeline_c128mkd4.txt"
symb_default_performances = get_performance_from_console_output("data/results/pipeline_ada_75_500_10.txt")
sklearn_default_performances = get_performance_from_console_output("data/results/pipeline_ada_default.txt")

## 2.2 Comparing Results
We compare results by number of times one's average cross-validation performance is better (first three columns) and by their loss as compared to the best found result in the original set of experiments (last column).

In [16]:
import pandas as pd
import numpy as np

methods = ['Symbolic', 'sklearn']
df = pd.DataFrame(np.zeros(shape=(len(methods), len(methods)+2)), columns = methods + ['loss', 'N'])
df.index = methods

# Calculate 'wins'
performances = list(zip(methods, [symb_default_performances, sklearn_default_performances]))
for (method, performance) in performances:
    for (method2, performance2) in performances:
        one_over_two = (performance.avg - performance2.avg) > 0
        df.loc[method][method2] = sum(one_over_two)

# Calculate loss        
for (method, performance) in performances:
    loss_sum = 0
    for i, row in performance.iterrows():
        best_score = adaboost_results[adaboost_results.task_id == row.name].predictive_accuracy.max()
        loss = best_score - row.avg
        if loss < 0:
            print('{} outperformed best on task {} by {}'.format(method, row.name, loss))
        loss_sum += loss
    df.loc[method]['loss'] = loss_sum
    df.loc[method]['N'] = len(performance)
    
df

Symbolic outperformed best on task 53 by -0.0035971316526609565
Symbolic outperformed best on task 20 by -0.0015000000000000568
Symbolic outperformed best on task 16 by -0.0030000000000001137
Symbolic outperformed best on task 14 by -0.0045000000000000595
sklearn outperformed best on task 10101 by -0.0013314594594595608


,Symbolic,sklearn,loss,N
Symbolic,0.0,43.0,1.900419,59.0
sklearn,14.0,0.0,19.729290,93.0


 Symbolic defaults take far longer to train, and I could not reasonably evaluate all datasets.

# 3. Random Forest

## 3.1 Loading Results

In [17]:
# The grid search result from Jan.
rfc_results = load_random_search_results('randomforest')

# results currently still stored in log. should be aggregated to single file..
# "data/results/pipeline_c128mkd4.txt"
symb_default_performances = get_performance_from_csv("data/results/ppp_rf_p027.txt")
sklearn_default_performances = get_performance_from_csv("data/results/ppp_rfc_default.txt")

## 3.2 Comparing Results
We compare results by number of times one's average cross-validation performance is better (first three columns) and by their loss as compared to the best found result in the original set of experiments (last column).

In [37]:
import pandas as pd
import numpy as np

methods = ['Symbolic', 'sklearn']
df = pd.DataFrame(np.zeros(shape=(len(methods), len(methods)+2)), columns = methods + ['loss', 'N'])
df.index = methods

# Calculate 'wins'
performances = list(zip(methods, [symb_default_performances, sklearn_default_performances]))
for (method, performance) in performances:
    for (method2, performance2) in performances:
        one_over_two = (performance.avg - performance2.avg) > 0
        df.loc[method][method2] = sum(one_over_two)

# Calculate loss        
for (method, performance) in performances:
    loss_sum = 0
    for i, row in performance.iterrows():
        best_score = rfc_results[rfc_results.task_id == row.name].predictive_accuracy.max()
        if np.isnan(best_score):
            continue
        loss = best_score - row.avg
        if loss < 0:
            print('{} outperformed best on task {} by {}'.format(method, row.name, loss))
        print(method, row.name, loss, best_score, row.avg)
        loss_sum += loss
    df.loc[method]['loss'] = loss_sum
    df.loc[method]['N'] = len(performance)
    
df

Symbolic 3543 0.0020000000000000018 0.988 0.986
Symbolic 3567 0.0 1.0 1.0
Symbolic 125920 0.04999999999999993 0.642 0.5920000000000001
Symbolic 125921 0.05800000000000005 0.756 0.698
Symbolic 3913 0.024935933236574614 0.856322 0.8313860667634254
Symbolic 9980 0.011111037037036975 0.924074 0.912962962962963
Symbolic 14968 0.014814777777777732 0.837037 0.8222222222222223
Symbolic 3494 0.00907259740259736 0.98917 0.9800974025974026
Symbolic 3492 0.0 1.0 1.0
Symbolic 9946 0.008748205513784302 0.968366 0.9596177944862156
Symbolic 9950 0.017519055656382454 0.910683 0.8931639443436176
Symbolic 9971 0.015237874926943284 0.737564 0.7223261250730567
Symbolic outperformed best on task 3512 by -0.0033336666666667902
Symbolic 3512 -0.0033336666666667902 0.988333 0.9916666666666668
Symbolic 3493 0.11316939890710387 1.0 0.8868306010928961
Symbolic 11 0.03533681515616982 0.8976 0.8622631848438301
Symbolic 3561 0.023851473222124664 0.693452 0.6696005267778753
Symbolic 41 0.016025015345268567 0.947291 0

,Symbolic,sklearn,loss,N
Symbolic,0.0,39.0,1.019086,95.0
sklearn,47.0,0.0,1.143054,95.0


The default of `max_features=0.271` did outperform the default of `sqrt(p)` in terms of loss. However, it is not a clear better default. The default of `sqrt(p)` is still better more often and scales much better with the amount of features in a dataset in terms of runtime.

----
**note**: Everything below is scratchpad and should be ignored

----